In [1]:
from auto_ml_regression.automl_pipeline import AutoMLPipeline
import pandas as pd

data = pd.read_csv('../../datasets/insurance_dataset/insurance.csv')
target_col="charges"
N_SEGMENTS = 2

cat_features = data.select_dtypes(include='object').columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list()
num_features.remove(target_col)

In [2]:
# Инициализация
pipeline = AutoMLPipeline(
    num_features=num_features,
    cat_features=cat_features,
    target_col=target_col,
    n_segments=N_SEGMENTS   # Можно указать любое количество сегментов
)

In [3]:
# Запуск полного пайплайна
pipeline.preprocess_data(data)

In [4]:
pipeline.segment_data_gmm()


In [5]:
pipeline.train_classifier(n_trials=10)

/Users/andrejurkinair/Documents/diploma_practice/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-21 21:40:37,965] A new study created in memory with name: no-name-fe7940f6-9bc1-470a-9ca4-84f7293a2f50
[I 2025-05-21 21:40:38,140] Trial 0 finished with value: 0.9130877609913428 and parameters: {'iterations': 1414, 'learning_rate': 0.008627858034159533, 'l2_leaf_reg': 0.007873507032925525, 'random_strength': 0.6049765300105571, 'bagging_temperature': 9.927267588969011, 'border_count': 142, 'min_data_in_leaf': 76, 'leaf_estimation_iterations': 12, 'grow_policy': 'Depthwise', 'od_type': 'Iter', 'od_wait': 38, 'one_hot_max_size': 218, 'max_depth': 4}. Best is trial 0 with value: 0.9130877609913428.
[I 2025-05-21 21:40:38,480] Trial 1 finished with value: 0.9259887964691902 and parameters: {'it

Classification Report: TRAIN
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       681
           1       1.00      0.76      0.87       215

    accuracy                           0.94       896
   macro avg       0.97      0.88      0.91       896
weighted avg       0.95      0.94      0.94       896

ROC AUC = 0.9845063688829697
TEST ROC AUC = 0.8896


In [6]:
pipeline.train_regressors(model_type='huber', n_trials=50, cv=4)

[I 2025-05-21 21:40:41,679] A new study created in memory with name: no-name-58968624-bc9b-461c-a9f1-71e66dc5dc40
[I 2025-05-21 21:40:41,786] Trial 0 finished with value: -3319.142712867988 and parameters: {'epsilon': 7.885236400490485, 'alpha': 0.0685106610420834, 'fit_intercept': True}. Best is trial 0 with value: -3319.142712867988.
[I 2025-05-21 21:40:41,815] Trial 1 finished with value: -1159.1980798542813 and parameters: {'epsilon': 6.605559459276528, 'alpha': 0.0011769253133664872, 'fit_intercept': False}. Best is trial 1 with value: -1159.1980798542813.
[I 2025-05-21 21:40:41,836] Trial 2 finished with value: -3430.685535700217 and parameters: {'epsilon': 9.564352244207923, 'alpha': 0.09250569441968928, 'fit_intercept': True}. Best is trial 1 with value: -1159.1980798542813.
[I 2025-05-21 21:40:41,858] Trial 3 finished with value: -3042.15033740705 and parameters: {'epsilon': 2.1741395824449605, 'alpha': 0.04238660018086489, 'fit_intercept': True}. Best is trial 1 with value: -

In [7]:
pipeline.regressor.get_model_coefs()

,model_coef_segment_0,model_coef_segment_1
intercept_,0.000000,6957.671591
age,12564.869377,9442.897524
bmi,1776.291856,29959.026305
children,2549.191611,-127.444026
sex,-115.972513,188.637803
smoker,13249.317387,9092.292185
region,-177.112066,-688.248622


In [8]:
test_target_preds = pipeline.regressor.predict(pipeline.segmented_test[num_features+cat_features], 
                              segments=pipeline.segmented_test['segment_preds'],
                              )
test_target_true = pipeline.segmented_test.charges

In [9]:
from sklearn import metrics
r2 = metrics.r2_score(test_target_true, test_target_preds)
mse = metrics.mean_squared_error(test_target_true, test_target_preds)
mae = metrics.mean_absolute_error(test_target_true, test_target_preds)
mape = metrics.mean_absolute_percentage_error(test_target_true, test_target_preds)

print(f"R2 = {r2}")
print(f"MSE = {mse}")
print(f"RMSE = {mse**0.5}")
print(f"MAE = {mae}")
print(f"MAPE = {mape}")

R2 = 0.8050073615497906
MSE = 29185976.52361206
RMSE = 5402.404698244298
MAE = 2793.217590139512
MAPE = 0.19358343272587747


In [10]:
# Оценка качества
metrics_df = pipeline.evaluate_test()
metrics_df

,r2,mse,rmse,mae,mape
0,0.805007,2.918598e+07,5402.404698,2793.21759,0.193583


In [11]:
pipeline.segmenter